![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de popularidad en canción

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desarrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de popularidad en canción".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/competitions/miad-2025-12-prediccion-popularidad-en-cancion).

## Datos para la predicción de popularidad en cancion

En este proyecto se usará el conjunto de datos de datos de popularidad en canciones, donde cada observación representa una canción y se tienen variables como: duración de la canción, acusticidad y tempo, entre otras. El objetivo es predecir qué tan popular es la canción. Para más detalles puede visitar el siguiente enlace: [datos](https://huggingface.co/datasets/maharshipandya/spotify-tracks-dataset).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importación librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2025/main/datasets/dataTrain_Spotify.csv')
#dataTraining = pd.read_csv('E:/vhmen/Documents/MIAD - ANDES/9_Machine larning y LPN/3. Servicio en la nube/dataTrain_Spotify.csv')

dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2025/main/datasets/dataTest_Spotify.csv', index_col=0)
#dataTesting = pd.read_csv('E:/vhmen/Documents/MIAD - ANDES/9_Machine larning y LPN/3. Servicio en la nube/dataTest_Spotify.csv', index_col=0)

In [ ]:
# Visualización datos de entrenamiento
dataTraining.head()

In [ ]:
# Visualización datos de test
dataTesting.head()

In [ ]:
# Predicción del conjunto de test - acá se genera un número aleatorio como ejemplo
np.random.seed(42)
y_pred = pd.DataFrame(np.random.rand(dataTesting.shape[0]) * 100, index=dataTesting.index, columns=['Popularity'])

In [ ]:
# Guardar predicciones en formato exigido en la competencia de kaggle
y_pred.to_csv('test_submission_file.csv', index_label='ID')
y_pred.head()

## 1. Preprocesamiento de datos

Es esta sección se encuntrará la descripción y arreglo de los datos de entrenamiento para su posterior predicción.

In [ ]:
dataTraining.info()

In [ ]:
# la columna Unnamed: 0 presenta el mismo consecutivo que el index.
dataTraining.drop('Unnamed: 0',axis='columns', inplace=True)

In [ ]:
dataTraining.head(n=10)

La base de datos de entrenamiento presenta 20 columnas con 79800 filas, lo cual representa el 70% del total de registros, dejando un 30% de datos para el conjunto de prueba. Del total de variables 9 son variables con valores continuos con decimales, 5 son valores enteros, 5 son valores de texto y 1 es un valor booleano. La variable objetivo a predecir es 'popularity' la cual se busca predecir la popularidad de la cación a partir de las caracteristicas en la base de datos como  la duración, si contiene letras explicitas, si es bailable, entre otras.

In [ ]:
plt.hist(dataTraining['popularity'], bins=100, color='skyblue', edgecolor='black')
mensaje = ['Histograma de la variable popularidad']
plt.title(" ".join(mensaje))
plt.show()

In [ ]:
dataTraining['popularity'].describe()

La variable popularity presenta una distribución con dos picos, uno cercano al 22 y otro cercano al 43, esto sin tener en cuenta que un 14% de los datos presentan un valor de popularidad de cero, que representan a más de 11.000 canciones. Si bien el promedio de popularidad de las canciones es de 33 puntos, esta media presenta una desviación estandar de 22 puntos por lo que se presenta una distribució aplanada. El valor del puntaje puede variar de 0 a 100, siendo 100 una canción con alta popularidad y 0 una canción con baja popularidad. Dado que se tiene una variable con un valor numerico de 0 a 100 se recomendaria inicialmente realizar una predicción mediante regresión, sin embargo se van a considerar más modelos. No se considera ajustar los valores de cero de la distribución debido a que solo representan un 14% de los datos los cuales si pueden dar indicios de la nula popularidad de una cación.

Hay 25.775 artistas, 114 generos, 37.315 nombres de albunes diferentes, 55.767 nombres de canciones diferentes, se propone incialmente no tener en cuenta estas variables debido a que se necesita realizar un proceso de segmentación de texto par apoder generalizar los artistas, generos, nombres de albunes y nombres de canciones de tal forma que se puede ver la influencia de cada palabra encontrada en la popularidad de la cancion, asi mismo seria computacionalmente pesado el procesar cada ramificación de los más de 25 mil artistas o incluso el nombre de los albunes o nombres de canciones.

In [ ]:
df_num = dataTraining.select_dtypes(include=np.number)

In [ ]:
sns.pairplot(df_num, kind="scatter")
plt.show()

In [ ]:
corr_Mis=df_num.corr()
corr_Mis.style.background_gradient(cmap='coolwarm')

Al revisar las variables númericas de la base de datos, las variables de tempo y danceability presentan distribuciones en forma de campana, de resto son distribuciones sesegadas o uniformes. Al comparar las variables con la variable a predecir, estas no presentan correlación significativa con la variable popularity, asi mismo, de las correlaciones entre todas las variables númericas solo la relación loudness-energy parece tener una correlación positiva en donde al aumentar la sonoridad aumenta la energia presentando un coeficiente de correlación del 76%. Por otro lado, otra correlación potencial puede ser la de acousticness-energy la cual es negativa en donde a mayor confianza de que la pista sea acustica menor es la energia, esto se representa con un coeficiente de correlación del -73% sin embargo, al revisar el grafico de puntos esta correlación negativa no es muy clara.

A partir de lo anterior se puede mencionar de manera general que las variables numericas explicativas del modelo a proponer no presentan correlaciones significativas y que estas tampoco presentan correlaciones con la variable dependiente.

In [ ]:
dataTraining['explicit']=np.where(dataTraining['explicit']==True,1,0)
dataTesting['explicit']=np.where(dataTesting['explicit']==True,1,0)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
# selección de train y test y variables dependientes e independientes
y_total=dataTraining['popularity']
x_total=dataTraining[['duration_ms','explicit','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature']]

xTrain, xTest, yTrain, yTest = train_test_split(x_total, y_total, test_size=0.30, random_state=42)

In [ ]:
yTrain_1=dataTraining['popularity']
xTrain_1=dataTraining[['duration_ms','explicit','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature']]
xTest_1=dataTesting[['duration_ms','explicit','danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature']]

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
s = StandardScaler()
xTrain_s = s.fit_transform(xTrain)
xTrain_1_s = s.fit_transform(xTrain_1)

In [ ]:
xTest_s = s.transform(xTest)
xTest_1_s = s.transform(xTest_1)

## 2. Calibración y entrenamiento del modelo

#### 2.1 Arboles de decisión

In [ ]:
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.model_selection import cross_val_score

In [ ]:
# Lista de valores para calibrar el criterio de parada de máxima profundidad (max_depth)
max_depth_range = range(1, 21)

# Lista para guardar los valores del RMSE para cada valor de máxima profundidad (max_depth)
MSE_scores_DT = []

# Loop para obtener el desempeño del modelo de acuerdo con la máxima profundidad
for depth in max_depth_range:
    # Definición del árbol de decisión usando DecisionTreeClassifier de la libreria sklearn
    clf = DecisionTreeRegressor(max_depth=depth, random_state=1)
    MSE_scores_DT.append(cross_val_score(clf,  xTrain_s, yTrain, cv=5, scoring='neg_mean_squared_error').mean())

In [ ]:
plt.plot(max_depth_range, MSE_scores_DT)
plt.xlabel('max_depth')
plt.ylabel('nMSE')

In [ ]:
np.mean(np.sqrt(np.abs(MSE_scores_DT)))

In [ ]:
sorted(zip(MSE_scores_DT, max_depth_range),reverse=True)[0]

In [ ]:
clf_dt = DecisionTreeRegressor(max_depth=10, random_state=1)
clf_dt.fit(xTrain_s,yTrain)

yPred_dt=clf_dt.predict(xTest_s)

In [ ]:
mse_dt_adj=mean_squared_error(yTest, yPred_dt, squared=False)
#mse_dt_adj=np.sqrt(mean_squared_error(yTest, yPred_dt))
mse_dt_adj

#### 2.1 Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
depth_range = range(1, 70)
MSE_scores_RF_md = []

for estimator in depth_range:
    rf = RandomForestRegressor(max_depth=estimator, random_state=1, n_jobs=-1)
    scores = cross_val_score(rf, xTrain_s, yTrain, cv=5, scoring='neg_root_mean_squared_error')
    MSE_scores_RF_md.append(scores.mean())


plt.plot(depth_range, MSE_scores_RF_md)
plt.xlabel('n_estimators')
plt.ylabel('nMSE')
plt.show()

In [ ]:
sorted(zip(MSE_scores_RF_md, depth_range),reverse=True)[0]

In [ ]:
# Creación de lista de valores para iterar sobre diferentes valores de n_estimators
estimator_range = range(5, 150, 5)

# Definición de lista para almacenar la exactitud (accuracy) promedio para cada valor de n_estimators
MSE_scores_RF = []

# Uso de un 5-fold cross-validation para cada valor de n_estimators
for estimator in estimator_range:
    clf4 = RandomForestRegressor(n_estimators=estimator, random_state=1, n_jobs=-1)
    MSE_scores_RF.append(cross_val_score(clf4,  xTrain_s, yTrain, cv=5, scoring='neg_mean_squared_error').mean())


plt.plot(estimator_range, MSE_scores_RF)
plt.xlabel('n_estimators')
plt.ylabel('nMSE')
plt.show()

In [ ]:
sorted(zip(MSE_scores_RF, estimator_range),reverse=True)[0]

In [ ]:
# Creación de lista de valores para iterar sobre diferentes valores de max_features
feature_cols=xTrain.columns
feature_range = range(1, len(feature_cols)+1)

# Definición de lista para almacenar la exactitud (accuracy) promedio para cada valor de max_features
MSE_scores_RF_1 = []

# Uso de un 10-fold cross-validation para cada valor de max_features
for feature in feature_range:
    clf5 = RandomForestRegressor(n_estimators=200, max_features=feature, random_state=1, n_jobs=-1)
    MSE_scores_RF_1.append(cross_val_score(clf5, xTrain_s, yTrain, cv=5, scoring='neg_mean_squared_error').mean())



plt.plot(feature_range, MSE_scores_RF_1)
plt.xlabel('n_features')
plt.ylabel('nMSE')
plt.show()

In [ ]:
sorted(zip(MSE_scores_RF_1, feature_range),reverse=True)[0]

In [ ]:
clf_rf = RandomForestRegressor(n_estimators=145, max_features=5, max_depth=52, random_state=1, n_jobs=-1)
clf_rf.fit(xTrain_s,yTrain)

yPred_rf=clf_rf.predict(xTest_s)

In [ ]:
mse_rf_adj=mean_squared_error(yTest, yPred_rf, squared=False)
#mse_rf_adj=np.sqrt(mean_squared_error(yTest, yPred_rf))
mse_rf_adj

#### 2.1 XGBoost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
learning_rate_op=[0.01, 0.05, 0.1, 0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7]

# Definición de lista para almacenar la exactitud (accuracy) promedio para cada valor de max_features
MSE_scores_XGB = []

# Uso de un 10-fold cross-validation para cada valor de max_features
for LRate in learning_rate_op:
    clf7 = XGBRegressor(learning_rate=LRate)
    MSE_scores_XGB.append(cross_val_score(clf7, xTrain_s, yTrain,cv=5, scoring='neg_mean_squared_error').mean())

# Gráfica del desempeño del modelo vs la cantidad de max_features
plt.plot(learning_rate_op, MSE_scores_XGB)
plt.xlabel('Learning_rate')
plt.ylabel('nMSE')
plt.show()

In [ ]:
sorted(zip(MSE_scores_XGB, learning_rate_op),reverse=True)[0]

In [ ]:
gamma_op=[0, 0.1, 0.5, 1]

# Definición de lista para almacenar la exactitud (accuracy) promedio para cada valor de max_features
MSE_scores_XGB_g = []

# Uso de un 10-fold cross-validation para cada valor de max_features
for gam in gamma_op:
    clf7 = XGBRegressor(gamma=gam)
    MSE_scores_XGB_g.append(cross_val_score(clf7, xTrain_s, yTrain,cv=5, scoring='neg_mean_squared_error').mean())


# Gráfica del desempeño del modelo vs la cantidad de max_features
plt.plot(gamma_op, MSE_scores_XGB_g)
plt.xlabel('Gamma')
plt.ylabel('nMSE')
plt.show()

In [ ]:
sorted(zip(MSE_scores_XGB_g, gamma_op),reverse=True)[0]

In [ ]:
colsample_bytree_op=[0.2, 0.4, 0.6, 0.8, 1.0]

# Definición de lista para almacenar la exactitud (accuracy) promedio para cada valor de max_features
MSE_scores_XGB_c = []

# Uso de un 10-fold cross-validation para cada valor de max_features
for csbt in colsample_bytree_op:
    clf7 = XGBRegressor(colsample_bytree=csbt)
    MSE_scores_XGB_c.append(cross_val_score(clf7, xTrain_s, yTrain,cv=5, scoring='neg_mean_squared_error').mean())


# Gráfica del desempeño del modelo vs la cantidad de max_features
plt.plot(colsample_bytree_op, MSE_scores_XGB_c)
plt.xlabel('Colsample')
plt.ylabel('nMSE')
plt.show()

In [ ]:
sorted(zip(MSE_scores_XGB_c, colsample_bytree_op),reverse=True)[0]

In [ ]:
clf_xgb_Adj = XGBRegressor(learning_rate=0.35,gamma=1,colsample_bytree=1)
clf_xgb_Adj.fit(xTrain_s, yTrain)
y_pred_XGB_adj = clf_xgb_Adj.predict(xTest_s)

In [ ]:
mse_XGb_adj=mean_squared_error(yTest, y_pred_XGB_adj, squared=False)
#mse_XGb_adj=np.sqrt(mean_squared_error(yTest, y_pred_XGB_adj))
mse_XGb_adj

In [ ]:
# Celda 8
fig = plt.figure()
ax = fig.add_axes([0, 0, 1, 1])
ax.set_title("Comparación de MSE entre los modelos")

ejeX = ['Decision Tree Ajustado','Random Forest Ajustado' ,'XGBoost ajustado']
ejeY = [mse_dt_adj,mse_rf_adj,mse_XGb_adj]

ax.bar(ejeX, ejeY)

def addlabels(x, y, plotP):
    for i in range(len(x)):
        plotP.text(i, y[i], f"{y[i]:.3f}", ha='center', va='bottom')

addlabels(ejeX, ejeY, plt)

plt.show()

Con base en el MSE se selecciona el modelo de Random Forest Ajustado

In [ ]:
# sacar predicción para Kaggle
clf_rf = RandomForestRegressor(n_estimators=145, max_features=5, max_depth=52, random_state=1, n_jobs=-1)
clf_rf.fit(xTrain_1_s,yTrain_1)

yPred_rf_Kg=clf_rf.predict(xTest_1_s)

In [ ]:
# Guardar predicciones en formato exigido en la competencia de kaggle
y_pred_kg = pd.DataFrame(yPred_rf_Kg, index=dataTesting.index, columns=['Popularity'])
y_pred_kg.to_csv('E:/vhmen/Documents/MIAD - ANDES/9_Machine larning y LPN/3. Servicio en la nube/test_submission_file_grupo_29_2.csv', index_label='ID')
y_pred_kg.head()

## 3. Disponibilización del modelo

In [5]:
import joblib

import os
os.chdir('..')

from flask import Flask
from flask_restx import Api, Resource, fields, reqparse

In [ ]:
# entrenamiento del modelo seleccionado
clf_rf = RandomForestRegressor(n_estimators=145, max_features=5, max_depth=52, random_state=1, n_jobs=-1)
cross_val_score(clf_rf,xTrain_1,yTrain_1, cv=10)

In [ ]:
clf_rf.fit(xTrain_1,yTrain_1)

In [ ]:
joblib.dump(clf_rf, 'E:/vhmen/Documents/MIAD - ANDES/9_Machine larning y LPN/3. Servicio en la nube/train_music_P2.pkl', compress=3)

In [ ]:
def predict_m(duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature):

    clf = joblib.load('E:/vhmen/Documents/MIAD - ANDES/9_Machine larning y LPN/3. Servicio en la nube/train_music_P2.pkl')
    
    datatrack={'duration_ms':[duration_ms],
          'explicit':[explicit],
          'danceability':[danceability],
          'energy':[energy],
          'key':[key],
          'loudness':[loudness],
          'mode':[mode],
          'speechiness':[speechiness],
          'acousticness':[acousticness],
          'instrumentalness':[instrumentalness],
          'liveness':[liveness],
          'valence':[valence],
          'tempo':[tempo],
          'time_signature':[time_signature]}
    datatrack = pd.DataFrame(datatrack)

    # Make prediction
    p1 = clf.predict(datatrack)

    return p1


app = Flask(__name__)

# Definición API Flask
api = Api(
    app, 
    version='1.0', 
    title='Popularity track Prediction API',
    description='Popularity track prediction API')

ns = api.namespace('predict', 
     description='Regression to find the Popularity of a Song')

# Definición argumentos o parámetros de la API
parser = reqparse.RequestParser()
parser.add_argument(
    'duration_ms', 
    type=int, 
    required=True, 
    help='Duration of a song in ms', 
    location='args')
parser.add_argument(
    'explicit', 
    type=int, 
    required=True, 
    help='Is 1 if the song have a explicit lirycs', 
    location='args')
parser.add_argument(
    'danceability', 
    type=float, 
    required=True, 
    help='A value of 0.0 is less danceable and 1.0 is totally danceable.', 
    location='args')
parser.add_argument(
    'energy', 
    type=float, 
    required=True, 
    help='A value between 0.0 and 1.0 that represent the intensity and perceptual activity of the track', 
    location='args')
parser.add_argument(
    'key', 
    type=int, 
    required=True, 
    help='The key in which the track is located', 
    location='args')
parser.add_argument(
    'loudness', 
    type=float, 
    required=True, 
    help='The overall loudness of the track in decibels', 
    location='args')
parser.add_argument(
    'mode', 
    type=int, 
    required=True, 
    help='Indicates the mode (major or minor) of the track, 1: major, 0:minor', 
    location='args')
parser.add_argument(
    'speechiness', 
    type=float, 
    required=True, 
    help='Detects the presence of spoken words in the track. 1 indicate that the track is entirely spoken and 0.33 indicate a mix between music and speak.', 
    location='args')
parser.add_argument(
    'acousticness', 
    type=float, 
    required=True, 
    help='A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1 is a high confidence that the track is acoustic. ', 
    location='args')
parser.add_argument(
    'instrumentalness', 
    type=float, 
    required=True, 
    help='Predicts if a track contains no vocals. 1 means that the track have a high probability to be instrumental', 
    location='args')
parser.add_argument(
    'liveness', 
    type=float, 
    required=True, 
    help='Detects the presence of an audience in the recording. 1 means that the track is live record.', 
    location='args')
parser.add_argument(
    'valence', 
    type=float, 
    required=True, 
    help='A measure from 0 to 1 that describes the musical positivity of the track, high values mean a positive track', 
    location='args')
parser.add_argument(
    'tempo', 
    type=float, 
    required=True, 
    help='The estimated tempo of the track in beats per minute (BPM)', 
    location='args')
parser.add_argument(
    'time_signature', 
    type=int, 
    required=True, 
    help='An estimated time signature, indicating how many beats there are in each measure. Values between 3 to 7 that means 3/4, 4/4...', 
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.String,
})


@ns.route('/')
class PopularityTrackApi(Resource):
    @ns.expect(parser)
    #@ns.doc(parser=parser)
    @ns.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()

        try: 
            prediction = predict_m(args['duration_ms'],
            args['explicit'],
            args['danceability'],
            args['energy'],
            args['key'],
            args['loudness'],
            args['mode'],
            args['speechiness'],
            args['acousticness'],
            args['instrumentalness'],
            args['liveness'],
            args['valence'],
            args['tempo'],
            args['time_signature'])
            return {"result":prediction}, 200
        except Exception as e:
            return {'error': str(e)}, 400


app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)

In [ ]:
289160,False,0.409,0.197,0,-13.803,1,0.0294	,0.797000,0.000319,0.2670,0.0615,91.952,4

In [ ]:
http://localhost:5000/predict/?duration_ms=289160&explicit=0&danceability=0.409&energy=0.197&key=0&loudness=-13.803&mode=1&speechiness=0.0294&acousticness=0.797000&instrumentalness=0.000319&liveness=0.2670&valence=0.0615&tempo=91.952&time_signature=4